In [1]:

import numpy as np
from obspy.core.trace import Trace
import time
import os
import cupy as cp

In [ ]:
#select which gpu should be used when working with a cluster 
gpu_index = 1

In [6]:
window_size = 300
n_channels = 286
n_samples = 15000
n_features = 60
samples_per_second = 50
samples_within_average = 25
n_sub_samples = int(n_samples / samples_within_average)
delta = 1.0 / float(samples_per_second)
freq_min = 0.2
freq_max = 24.0
freq_min_space= 0.04
freq_max_space= 2
w0 =8

#scales should be loaded onto gpu 
with cp.cuda.Device(gpu_index):
    time_log = cp.logspace(cp.log10(freq_min), cp.log10(freq_max), 30)
    space_log = cp.logspace(cp.log10(freq_min_space), cp.log10(freq_max_space), 30)

data shape:  (286, 15000)
(286, 30000)


### Parameter for CWT that will be used later

window_size: the length of the cwt window in seconds

n_channels: the number of channels in the data

n_samples: the total samples in the data

n_features: number of features per sample after performing cwt 30 will be on the time axis 30 will be on the spacial axis

samples_per_second: samples taken per second at each channel

samples_within_average: the amount of cwt features that will be average per set in the final output

n_sub_samples: n_samples/samples_within_average amount of samples that will be in the final output

delta: 1/samples_per_second or the sample period

freq_min: min frequency allowed through bandpass filter

freq_max: max frequency allowed through bandpass filter

freq_min_space: min frequency used for the spatial scales of cwt

freq_max_space: max frequency used for spatial scales of cwt

time_scales: scales for time cwt

spatial_scales: scales for spatial cwt



### formula to calculate sacles from frequency where $ \lambda$ = frequency
$$ scale = \frac{\lambda(\omega_0 + \sqrt{2 + \omega_0})}{4\pi}  $$

### formula for angular frequencies
$$ w_k = \begin{cases}
   \frac{2 \pi k}{dt} & \text{ k $\le \frac{N}{2} $ }\\
   -\frac{2 \pi k}{dt} & \text{ k >$\ \frac{N}{2} $ }\\
    \end{cases}$$

### formula for morlet wavelet


$$ \psi_0(N) = \pi^{ -\frac{1}{4}} e^{iw_0N} e^{-n^2/2}
$$

### formula for morlet wavelet at a scale


$$ \psi_0(sw) = \pi^{ -\frac{1}{4}} H(w) e^{-(sw - w_0)^2/2}
$$

### Normalization term for wavelet

$$ \sqrt{ \frac{2\pi s}{dt}}$$

In [8]:
def cwt_time_vec(signal, log_space, omega_0, dt, space=False):
    with cp.cuda.Device(gpu_index):
        #signal shape is (channels , samples)
        #calculate the scales for cwt
        scales = (log_space*(omega_0 + cp.sqrt(2 + omega_0**2)))/ (4 * cp.pi)
        #calculate angular frequencies
        angular_freq = ((2*cp.pi * cp.arange(signal.shape[1])) / (signal.shape[1]* dt))
        # reflect the second half of the array
        angular_freq[(signal.shape[1] //2):] *= -1
        #H(w)
        heavy_step = angular_freq > 0

        #compute wavelet point
        wavelet = cp.zeros((signal.shape[1], scales.shape[0]), dtype=cp.complex128)
        wavelet[heavy_step,:]= np.sqrt((scales *2 * cp.pi)/ dt) * (cp.pi ** -.25) * cp.exp(-((cp.outer(angular_freq[heavy_step], scales) - omega_0)**2) / 2)

        #zero mean
        signal -= cp.mean(signal)

        fft_signal = cp.fft.fft(signal)

        wavelet = wavelet[:, cp.newaxis, :]
        signal = signal[cp.newaxis, :,:]
        result = cp.array((wavelet.shape[1], fft_signal.shape[1]), dtype=cp.complex128)
        multiplied = wavelet.T * fft_signal
        mempool = cp.get_default_memory_pool()

        if(not space):
            multiplied = multiplied.transpose(0,2,1)
            
        del scales 
        del angular_freq
        del wavelet
        del signal
        del fft_signal
        
        cp._default_memory_pool.free_all_blocks()

        inverse_start = time.time()
        result = cp.fft.ifft(multiplied, axis=1)
        inverse_end = time.time()

        return result

In [12]:
def cwt_space_vec(signal, log_space, omega_0, dt):
    with cp.cuda.Device(gpu_index):
        #calculate the scales for cwt
        scales = (log_space*(omega_0 + cp.sqrt(2 + omega_0**2)))/ (4 * cp.pi)
        #calculate angular frequencies
        angular_freq = ((2*cp.pi * cp.arange(signal.shape[1])) / (signal.shape[1]* dt))
        # reflect the second half of the array
        angular_freq[(signal.shape[1] //2):] *= -1
        #H(w)
        heavy_step = angular_freq > 0

        #compute wavelet point
        wavelet = cp.zeros((signal.shape[1], scales.shape[0]), dtype=cp.complex128)
        wavelet[heavy_step,:]= cp.sqrt((scales *2 * cp.pi)/ dt) * (cp.pi ** -.25) * cp.exp(-((cp.outer(angular_freq[heavy_step], scales) - omega_0)**2) / 2)

        #zero mean
        signal -= cp.mean(signal)


        fft_signal = cp.fft.fft(signal)

        wavelet = wavelet[:, cp.newaxis, :]
        signal = signal[cp.newaxis, :,:]

        multiplied = wavelet.T * fft_signal


        result = cp.array((wavelet.shape[1], fft_signal.shape[0]), dtype=cp.complex128)
        result = cp.fft.ifft(multiplied, axis=2)
        return result

## This transforms one window of data

In [10]:
def transform_window(data):
    """transform a window of data from stanford array

    Args:
    data (np.array(286, 15000)): window size should be 5 min/ 15000 samples

    Returns:
    cwt transformed data : np.array(286, 300,60) only samples from 250:7750 are used so result is 2.5 minutes long
    """
    with cp.cuda.Device(gpu_index):
        n_samples=data.shape[1]

        #take time derivative
        data_derivative = data[:, 1:] - data[:, :-1]

        # perform bandpass filter to remove high and low frequencies
        # TODO can this be done faster
        for channel in range(n_channels):
            trace = Trace(data=data_derivative[channel, :], header={'delta':1.0/float(samples_per_second),'sampling_rate':samples_per_second})
            trace = trace.filter("bandpass", freqmin= freq_min, freqmax= freq_max, corners=4, zerophase=True)
            data_derivative[channel, :] = trace.data

        #remove lazer drift/ remove median
        data_derivative = data_derivative - np.median(data_derivative, axis=0)
        data_derivative = cp.array(data_derivative)

        transformed_data = cp.empty((n_channels, n_samples-1, 60), dtype=cp.float64)

        
        transformed_data[:, : , :30] = cp.abs(cwt_time_vec(data_derivative, time_log, w0, delta).T)
        
        transformed_data[:, : , 30:] = cp.abs(cwt_space_vec(data_derivative.T, space_log, w0, delta).T)

        #average samples over 0.5 second intervals
        #start is at 250 since stanford windows start at 54.9 so add 5 seconds to start at the top of
        reshaped_data = cp.reshape(transformed_data[:, 250: 7750, :], (n_channels, 300, samples_within_average, 60))

        averaged_data = cp.mean(reshaped_data, axis=2)
        return averaged_data




In [ ]:
with cp.cuda.Device(gpu_index):
    dir = "./DAS_data/09"
    out_dir = "CWT_data/09"
    
    file_list = os.listdir(dir)
    sorted_list = sorted(file_list)
    

    count = 0
    for file in sorted_list:
        count +=1
        data = np.load(dir+ "/"+ file)
        start = time.time()
        result = transform_window(data)
        end = time.time()
        print(f'transform {count} took {end-start}')
        
        filename = file.split(".")[0]
        np.save(out_dir + "/" + "cwt_" + filename, result)
        
        